In [1]:
!python --version

Python 3.10.6


In [2]:
!pip install ultralytics==8.0.106

     ------------------------------------- 587.4/587.4 kB 30.3 kB/s eta 0:00:00
  Using cached torchvision-0.20.1-cp310-cp310-win_amd64.whl (1.6 MB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached scipy-1.14.1-cp310-cp310-win_amd64.whl (44.8 MB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached torch-2.5.1-cp310-cp310-win_amd64.whl (203.1 MB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ------------------------------------- 294.9/294.9 kB 33.6 kB/s eta 0:00:00
  Using cached pillow-11.0.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)
     ------------------------------------- 322.9/322.9 kB 21.5 kB/s eta 0:00:00
  Using cached matplotlib-3.10.0-cp310-cp310-win_amd64.whl (8.0 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     --------------------------------------- 71.9/71.9 kB 22.8 kB/s 


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install torch


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import ultralytics
ultralytics.__version__

WARNING  Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at 'C:\Users\PMLS\AppData\Roaming\Ultralytics\settings.yaml'


'8.0.106'

In [5]:
import torch
torch.__version__

'2.5.1+cpu'

In [6]:
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

# Detect, track and count Persons

In [7]:
%cd yolov8_DeepSORT

[WinError 2] The system cannot find the file specified: 'yolov8_DeepSORT'
c:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT


c:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\env\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [1]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

In [14]:
# Load a model
model = YOLO("best.pt")  # load a pretrained model (recommended for training)

results = model("image.png", save=True)

class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    probs = result.probs  # Class probabilities for classification outputs
    cls = boxes.cls.tolist()  # Convert tensor to list
    xyxy = boxes.xyxy
    xywh = boxes.xywh  # box with xywh format, (N, 4)
    conf = boxes.conf
    print(cls)
    for class_index in cls:
        class_name = class_names[int(class_index)]
        print("Class:", class_name)


image 1/1 C:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\image.png: 640x640 1 truck, 1228.9ms
Speed: 8.5ms preprocess, 1228.9ms inference, 57.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to c:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\runs\detect\predict5


[8.0]
Class: boat


# DeepSORT

In [17]:
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort
from deep_sort_pytorch.utils.tools import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

c:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\deep_sort\deep\feature_extractor.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

In [25]:
# Define the video path
video_path = '2252223-sd_640_360_30fps.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [26]:
frames = []

unique_track_ids = set()

In [27]:
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

        results = model(frame)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
         'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        # Draw person count on frame
        cv2.putText(og_frame, f"Car Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

        # Show the frame
        cv2.imshow("Video", og_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 17 cars, 3 buss, 152.9ms
Speed: 4.1ms preprocess, 152.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 18 cars, 2 buss, 170.2ms
Speed: 3.0ms preprocess, 170.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 18 cars, 2 buss, 275.0ms
Speed: 2.7ms preprocess, 275.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 18 cars, 2 buss, 167.7ms
Speed: 3.0ms preprocess, 167.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 15 cars, 3 buss, 161.4ms
Speed: 5.8ms preprocess, 161.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 16 cars, 2 buss, 128.9ms
Speed: 3.5ms preprocess, 128.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 17 cars, 2 buss, 169.0ms
Speed: 3.0ms preprocess, 169.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 18 cars, 3 buss, 170.4ms
Speed: 3.0ms prepr

In [1]:
!python predict.py 

Traceback (most recent call last):
  File "c:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\predict.py", line 3, in <module>
    import hydra
ModuleNotFoundError: No module named 'hydra'


In [3]:
!pip install hydra

  Using cached Hydra-2.5.tar.gz (82 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for hydra, since package 'wheel' is not installed.
  Running setup.py install for hydra: started
  Running setup.py install for hydra: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Running setup.py install for hydra did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running install
      C:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\env\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      copying src\hydra.py -> build\lib.win-amd64-cpython-310
      running build_ext
      building '_hydra' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-fai

In [6]:
%pip install gdown

!gdown "https://drive.google.com/uc?id=1UUZUS76ylH5RjMSc9s4rITSYDTG5AHt0&confirm=t"

  Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Downloading...
From: https://drive.google.com/uc?id=1UUZUS76ylH5RjMSc9s4rITSYDTG5AHt0&confirm=t
To: c:\MainDrive\Work\Projects\TrafficMonitoringSystem\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\test4.mp4

  0%|          | 0.00/7.96M [00:00<?, ?B/s]
  7%|▋         | 524k/7.96M [00:00<00:07, 1.05MB/s]
 13%|█▎        | 1.05M/7.96M [00:00<00:04, 1.62MB/s]
 20%|█▉        | 1.57M/7.96M [00:00<00:03, 2.00MB/s]
 26%|██▋       | 2.10M/7.96M [00:01<00:02, 2.07MB/s]
 33%|███▎      | 2.62M/7.96M [00:01<00:02, 1.86MB/s]
 39%|███▉      | 3.15M/7.96M [00:01<00:02, 1.86MB/s]
 46%|████▌     | 3.67M/7.96M [00:01<00:02, 2.00MB/s]
 53%|█████▎    | 4.19M/7.96M [00:02<00:01, 1.95MB/s]
 59%|█████▉    | 4.72M/7.96M [00:02<00:01, 2.13MB/s]
 66%|██████▌   | 5.24M/7.96M [00:02<00:01, 2.15MB/s]
 72%|███████▏  | 5.77M/7.96M [00:02<00:01, 2.04MB/s]
 79%|███████▉  | 6.29M/7.96M [00:03<00:0